# Model Comparision

In this git we will comapre models which we trained on similar data-set of knee-mri


## Model Used 
  
1. Mask-Rcnn
2. Yolo
3. Unet


## Evaluation Matrix

1. Dice Coefficient
2. Jaccard
3. Intresection over Union of the masks

In [1]:
import os
import sys
import glob
import matplotlib.pyplot as plt
from evaluation.evaluation_utils import *
import cv2


def get_parent_dir(n=1):
    """returns the n-th parent dicrectory of the current
    working directory"""
    current_path = os.getcwd()
    for _ in range(n):
        current_path = os.path.dirname(current_path)
    return current_path


src_path   = os.path.join("TrainYourOwnYOLO" , "2_Training", "src")
utils_path = os.path.join("TrainYourOwnYOLO" , "Utils")

sys.path.append(src_path)
sys.path.append(utils_path)




## Yolo Utils

1. Weights paths
2. Ground Truth

In [2]:
import argparse
from keras_yolo3.yolo import YOLO, detect_video, detect_webcam
from PIL import Image
from timeit import default_timer as timer
from utils import load_extractor_model, load_features, parse_input, detect_object
import test
import utils
import pandas as pd
import numpy as np
from Get_File_Paths import GetFileList
import random
from Train_Utils import get_anchors
from PIL import ImageDraw, Image

# Setting prediction environemtn (cpu or gpu)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Setting weights path
model_folder = f"TrainYourOwnYOLO/Data/Model_Weights/"
model_weights = os.path.join(model_folder, "checkpoint.h5")
model_classes = os.path.join(model_folder, "data_classes.txt")
anchors_path = os.path.join(src_path, "keras_yolo3", "model_data", "yolo_anchors.txt")
anchors = get_anchors(anchors_path)

# Input Images
train_imgaes_path = "dataset/mrcnn_format/test/"
input_images_paths = glob.glob(f"{train_imgaes_path}*.*g")
ground_truths = f"{train_imgaes_path}/yolo_points.txt"



yolo = YOLO(
        **{
            "model_path": model_weights,
            "anchors_path": anchors_path,
            "classes_path": model_classes,
            "score": 0.01,
            "gpu_num": 1,
            "model_image_size": (416, 416),
        }
    )

with open (ground_truths, "r") as myfile:
    data = myfile.read().splitlines()
    
yolo_groundTruths = dict()
for i in data:
    image, ROI = i.split(" ")
    yolo_groundTruths.update({image:ROI})


Instructions for updating:
Colocations handled automatically by placer.
TrainYourOwnYOLO/Data/Model_Weights/checkpoint.h5 model, anchors, and classes loaded in 5.03sec.


# Unet Prediction Fucntion





In [3]:
from unet.model import *

weights_path = "unet/weights/Unet_46_0.9041458964347839_0.05271167308092117.h5"
model = unet()
model.load_weights(weights_path)
unet_testing_images_folder = "/media/sohaib/additional_/DataScience/knee_mri/dataset/unet_format/*/"

def unet_prediction(image_path):
    '''
    
        Get Unet Predictions
        
        
        Params: image_path : (Str) Path of the image
        
        
        Returns: pred  : (np.asarray) Mask of the image
    
    '''
    
    # Preprocessing
    image_name = image_path.rsplit("/",1)[1]
    image_path = glob.glob(unet_testing_images_folder+ "images/" + image_name)[0]
    mask_path = glob.glob(unet_testing_images_folder+ "masks/" + image_name)[0]
    
    # Opening Image and mask
    image = Image.open(image_path).convert("L")
    mask = Image.open(mask_path).convert("L")
    
    # Resizing Image
    image = image.resize((256, 256))
    mask  = mask.resize((256, 256))
    
    # Converting Image object to numpy array
    image = np.asarray(image)
    mask = np.asarray(mask)
    
    
    # Expanding diminsion to give it to the model
    image = np.expand_dims(image , axis=2)
    image = np.expand_dims(image , axis=0)
    
    
    # Prediction
    pred = model.predict(image)
    pred = pred.squeeze(axis = 0).squeeze(axis = 2)
    pred = np.expand_dims(pred , axis=2)
    
    
    mask = np.expand_dims(mask , axis=2)
    
    # Getting Rectangle cordinates
    rect    = mask_to_rect(pred)
    gt_rect = mask_to_rect(mask)
    
    
    
    return cv2.cvtColor(pred, cv2.COLOR_GRAY2RGB), rect, cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB) , gt_rect





/home/sohaib/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


# Yolo Prediction Fucntion

In [4]:


def Yolo_prediction(image_path):
    '''
    
        Get the predictions from the yolo
        
        
        Params: Image_path : (Str) Path of the image
        
        Returns: ROI            : (list) Cordinates of the rectangle ground truth
        Returns: PredictedROI   : (list) Cordinates of the rectangle prediction
        Returns: Mask           : (np.asarray) Cordinates of the rectangle ground truth
        Returns: Predictedmask  : (np.asarray) Cordinates of the rectangle prediction
    
    '''
    
    global yolo, yolo_groundTruths
    
    prediction, image = detect_object(
            yolo,
            image_path,
            save_img=False,
            save_img_path=False,
            postfix="",
        )
    
    
    if not prediction: 
        prediction = [[0,0,0,0, 0, 0]]
    
    
    roi = [int(i) for i in yolo_groundTruths[os.getcwd()+ "/" +image_path].split(",")[0:4]]
    gt_mask          = make_mask(roi, image.shape)
    
    # Getting ROI from YOLO
    x1, y1, w, h, label, score =  prediction[0]
    predicted_roi = [x1, y1, w, h]

    # Making Mask from ROI
    image            = np.asarray(Image.open(image_path))
    predicted_mask   = make_mask(predicted_roi, image.shape)
        
        
    return roi, predicted_roi, gt_mask, predicted_mask
        





# Mrcnn Utils

In [5]:
sys.path.append("mrcnn_training/")
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf
import mrcnn.model as modellib
import glob
from predict_utils import predict
import time
import matplotlib.pyplot as plt
from PIL import Image


# Set CPU as available physical device



'''

Mention Same Config file here as you mentioned in training notebook

'''

from mrcnn.config import Config

class GenericConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """

    def __init__(self, classes, steps):
        self.NUM_CLASSES = classes +1
        self.STEPS_PER_EPOCH = steps
        super().__init__()


    NAME = "class"
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.1
    IMAGE_MAX_DIM=448
    IMAGE_MIN_DIM=384
    TRAIN_ROIS_PER_IMAGE=20
    DETECTION_NMS_THRESHOLD=0.1
    DETECTION_MAX_INSTANCES=10

    


weights_path = "/media/sohaib/additional_/maskrcnn/weights/class20210905T1457/mask_rcnn_class_0190.h5"
MODEL_DIR = "/".join(weights_path.split("/")[:-2])
config = GenericConfig(1,100)
config.display()

# Create model in inference mode
with tf.device("/cpu:0"):
    mrcnn_model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,config=config)

mrcnn_model.load_weights(weights_path, by_name=True)  
print("Weights loaded")
    

    
    
def predict_mrcnn(image_path):
    
    labels = [ "ROI"]
    out, r = predict(mrcnn_model, image_path, labels, 0.01)
    return r['rois'][0], cv2.cvtColor(r['masks'].astype(np.uint8), cv2.COLOR_GRAY2RGB)
        
    



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        10
DETECTION_MIN_CONFIDENCE       0.1
DETECTION_NMS_THRESHOLD        0.1
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  448
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  384
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [448 448   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE              

In [ ]:
YOLO_jacard_coef, YOLO_IOU, YOLO_dice_coef = [], [], []
UNET_jacard_coef, UNET_IOU, UNET_dice_coef = [], [], []
MRCNN_jacard_coef, MRCNN_IOU, MRCNN_dice_coef = [], [], []
images_processed = []

for i, img_path in enumerate(input_images_paths):
    
    
    try:
        print(img_path)
        print("Yolo Logs")
        yolo_gt_roi, yolo_roi, yolo_gt_mask, yolo_mask = Yolo_prediction(img_path)
        print("\n")


        unet_mask, unet_rect, unet_gt_mask, unet_gt_rect = unet_prediction(img_path)


        maskrcnn_roi, maskrcnn_mask = predict_mrcnn(img_path)


        # Converting Rectangle to polygon
        yolo_gt_poly, yolo_poly = rect_to_poly(yolo_gt_roi), rect_to_poly(yolo_roi)
        unet_gt_poly, unet_poly = rect_to_poly(unet_gt_rect), rect_to_poly(unet_rect)
        maskrcnn_gt_poly        = rect_to_poly(maskrcnn_roi)


        # Calculating Intersection over union
        yolo_iou     = polygonIOU(yolo_poly, yolo_gt_poly)
        unet_iou     = polygonIOU(unet_poly, unet_gt_poly)
        maskrcnn_iou = polygonIOU(maskrcnn_gt_poly, yolo_gt_poly)
        print(f"yoloIOU {yolo_iou},  unet_iou {unet_iou} maskrcnn_iou {maskrcnn_iou}")


        # Calculating Jaccard
        yolo_jaccard_coef  = jaccard(yolo_mask, yolo_gt_mask)
        unet_jaccard_coef  = jaccard(unet_mask, unet_gt_mask)
        mrcnn_jaccard_coef = jaccard(maskrcnn_mask, yolo_gt_mask)
        print(f"yolo_jaccard_coef {yolo_jaccard_coef},  unet_jaccard_coef {unet_jaccard_coef} mrcnn_jaccard_coef {mrcnn_jaccard_coef}")


        # Calcualting Dice Coeffecient
        yolo_dice_c = dice(yolo_mask[:,:,0].astype(bool), yolo_gt_mask[:,:,0].astype(bool))
        unet_dice_c = dice(unet_mask[:,:,0].astype(bool), unet_gt_mask[:,:,0].astype(bool))
        mrcnn_dice_c = dice(maskrcnn_mask[:,:,0].astype(bool), yolo_gt_mask[:,:,0].astype(bool))
        print(f"yolo_dice_c {yolo_dice_c},  unet_dice_c {unet_dice_c} mrcnn_dice_c {mrcnn_dice_c}")
    
    except:
        pass
    
    else:
        
        # Saving Image which is processed
        images_processed.append(img_path.rsplit("/",1)[1])
        
        # Appending Yolo Results
        YOLO_jacard_coef.append(yolo_jaccard_coef)
        YOLO_IOU.append(yolo_iou)
        YOLO_dice_coef.append(yolo_dice_c)


        # Appending Unet Results
        UNET_jacard_coef.append(unet_jaccard_coef)
        UNET_IOU.append(unet_iou)
        UNET_dice_coef.append(unet_dice_c)
        
        # Appending MRCNN Results
        MRCNN_jacard_coef.append(mrcnn_jaccard_coef)
        MRCNN_IOU.append(maskrcnn_iou)
        MRCNN_dice_coef.append(mrcnn_dice_c)



dataset/mrcnn_format/test/788123-5_7.jpg
Yolo Logs
(416, 416, 3)
Found 2 boxes for img
background 0.07 (64, 43) (211, 185)
background 0.85 (80, 89) (239, 227)
Time spent: 0.025sec


yoloIOU 0.43139320516197,  unet_iou 0.837037037037037 maskrcnn_iou 0.7782946266716831
yolo_jaccard_coef 0.44386572129439833,  unet_jaccard_coef 0.8170004688860607 mrcnn_jaccard_coef 0.677536231884058
yolo_dice_c 0.6148296406628189,  unet_dice_c 0.8992848189928482 mrcnn_dice_c 0.8077753779697624
dataset/mrcnn_format/test/786061-5_16.jpg
Yolo Logs
(416, 416, 3)
Found 1 boxes for img
background 0.94 (79, 58) (241, 210)
Time spent: 0.025sec


yoloIOU 0.6503012720374917,  unet_iou 0.8407337919533041 maskrcnn_iou 0.5304347826086957
yolo_jaccard_coef 0.6918050226055241,  unet_jaccard_coef 0.8307657623807019 mrcnn_jaccard_coef 0.7891889132765045
yolo_dice_c 0.8178306759487985,  unet_dice_c 0.9075609555865692 mrcnn_dice_c 0.8821750542051808
dataset/mrcnn_format/test/808420-5_9.jpg
Yolo Logs
(416, 416, 3)
Found 3 box

yoloIOU 0.26865286089464846,  unet_iou 0.5190198596976583 maskrcnn_iou 0.40009521542489884
yolo_jaccard_coef 0.19399442781962645,  unet_jaccard_coef 0.5582662245381336 mrcnn_jaccard_coef 0.7039950449055435
yolo_dice_c 0.32495030680148645,  unet_dice_c 0.7165222678218575 mrcnn_dice_c 0.8262876667514812
dataset/mrcnn_format/test/784261-5_9.jpg
Yolo Logs
(416, 416, 3)
Found 1 boxes for img
background 0.84 (89, 75) (221, 247)
Time spent: 0.022sec


yoloIOU 0.7649721939374952,  unet_iou 0.7833869036364782 maskrcnn_iou 0.8372502937720329
yolo_jaccard_coef 0.731641049526471,  unet_jaccard_coef 0.8091095890410959 mrcnn_jaccard_coef 0.7009025688498033
yolo_dice_c 0.8450262249518089,  unet_dice_c 0.8944837769280278 mrcnn_dice_c 0.8241536954392076
dataset/mrcnn_format/test/785277-5_1.jpg
Yolo Logs
(416, 416, 3)
Found 2 boxes for img
background 0.01 (123, 124) (292, 302)
background 0.19 (84, 101) (236, 268)
Time spent: 0.025sec


yoloIOU 0.40386218393998335,  unet_iou 0.8796837944664032 maskrcnn_i

yoloIOU 0.5898829197243852,  unet_iou 0.8501882732651963 maskrcnn_iou 0.5066754475243969
yolo_jaccard_coef 0.54212299915754,  unet_jaccard_coef 0.8698462864530958 mrcnn_jaccard_coef 0.8603651987110634
yolo_dice_c 0.703086588363835,  unet_dice_c 0.9303933620159803 mrcnn_dice_c 0.9249422632794457
dataset/mrcnn_format/test/785645-5_3.jpg
Yolo Logs
(416, 416, 3)
Found 3 boxes for img
background 0.02 (76, 160) (244, 300)
background 0.04 (58, 10) (211, 163)
background 0.99 (85, 89) (228, 234)
Time spent: 0.027sec


yoloIOU 0.4198291167332446,  unet_iou 0.8338382938276006 maskrcnn_iou 0.9012595185679743
yolo_jaccard_coef 0.33211191592103395,  unet_jaccard_coef 0.83100006277858 mrcnn_jaccard_coef 0.7011444425177735
yolo_dice_c 0.49862464549971214,  unet_dice_c 0.9077007474456559 mrcnn_dice_c 0.8243208806890576
dataset/mrcnn_format/test/784261-5_16.jpg
Yolo Logs
(416, 416, 3)
Found 3 boxes for img
background 0.01 (49, 36) (226, 194)
background 0.02 (126, 38) (281, 194)
background 0.93 (80, 93) 

yoloIOU 0.27513678565339367,  unet_iou 0.7377965470127934 maskrcnn_iou 0.8125323872922231
yolo_jaccard_coef 0.34215467143495754,  unet_jaccard_coef 0.8297602151340853 mrcnn_jaccard_coef 0.8347520145272954
yolo_dice_c 0.5098587796429523,  unet_dice_c 0.9069606042049397 mrcnn_dice_c 0.909934430285785
dataset/mrcnn_format/test/786061-5_21.jpg
Yolo Logs
(416, 416, 3)
Found 2 boxes for img
background 0.06 (7, 81) (166, 232)
background 0.23 (98, 84) (225, 233)
Time spent: 0.025sec


dataset/mrcnn_format/test/786061-5_9.jpg
Yolo Logs
(416, 416, 3)
Found 2 boxes for img
background 0.02 (74, 13) (233, 166)
background 0.79 (90, 82) (226, 230)
Time spent: 0.025sec


yoloIOU 0.28284921990980644,  unet_iou 0.7954971857410882 maskrcnn_iou 0.5394231200562226
yolo_jaccard_coef 0.34710970335627706,  unet_jaccard_coef 0.8024878640776699 mrcnn_jaccard_coef 0.7520800367515696
yolo_dice_c 0.515339919965635,  unet_dice_c 0.8904224877966672 mrcnn_dice_c 0.8584996358339403
dataset/mrcnn_format/test/804892-5_1

yoloIOU 0.5007623682076237,  unet_iou 0.7618166626879923 maskrcnn_iou 0.8914900860880984
yolo_jaccard_coef 0.4417749448394214,  unet_jaccard_coef 0.7607059270036055 mrcnn_jaccard_coef 0.6846451612903226
yolo_dice_c 0.6128209488182282,  unet_dice_c 0.8640919705406862 mrcnn_dice_c 0.8128063725490197
dataset/mrcnn_format/test/790941-5_5.jpg
Yolo Logs
(416, 416, 3)
Found 1 boxes for img
background 0.99 (80, 107) (241, 250)
Time spent: 0.022sec


yoloIOU 0.8405223585278987,  unet_iou 0.853201684332801 maskrcnn_iou 0.5855097003788313
yolo_jaccard_coef 0.7354681069958847,  unet_jaccard_coef 0.8575489819439108 mrcnn_jaccard_coef 0.7807066741447
yolo_dice_c 0.8475731752500927,  unet_dice_c 0.9233123759099934 mrcnn_dice_c 0.8768503937007874
dataset/mrcnn_format/test/786518-5_10.jpg
Yolo Logs
(416, 416, 3)
Found 2 boxes for img
background 0.01 (59, 76) (212, 214)
background 0.83 (86, 111) (236, 259)
Time spent: 0.026sec


yoloIOU 0.4877454911280563,  unet_iou 0.9673328738800827 maskrcnn_iou 0.499

...

yoloIOU 0.75488334237656,  unet_iou 0.8375359342915811 maskrcnn_iou 0.8427070795491848
yolo_jaccard_coef 0.7749527822170565,  unet_jaccard_coef 0.8467326063065348 mrcnn_jaccard_coef 0.7963714490331821
yolo_dice_c 0.8732094622247688,  unet_dice_c 0.9170061799038681 mrcnn_dice_c 0.8866445182724253
dataset/mrcnn_format/test/805533-5_20.jpg
Yolo Logs
(416, 416, 3)
Found 2 boxes for img
background 0.02 (100, 140) (255, 273)
background 0.85 (97, 99) (221, 225)
Time spent: 0.025sec


dataset/mrcnn_format/test/813989-5_1.jpg
Yolo Logs
(416, 416, 3)
Found 3 boxes for img
background 0.02 (28, 26) (192, 196)
background 0.07 (51, 86) (222, 234)
background 0.26 (77, 123) (244, 296)
Time spent: 0.026sec


yoloIOU 0.2817302219692658,  unet_iou 0.46096788783355946 maskrcnn_iou 0.5301082768699349
yolo_jaccard_coef 0.3130290160672389,  unet_jaccard_coef 0.5725662077906044 mrcnn_jaccard_coef 0.3091710427606902
yolo_dice_c 0.47680441519078964,  unet_dice_c 0.7281934521472875 mrcnn_dice_c 0.472315736695079

yoloIOU 0.7679507385551442,  unet_iou 0.8846683581007612 maskrcnn_iou 0.6820768759303039
yolo_jaccard_coef 0.7403846153846154,  unet_jaccard_coef 0.8770867303362062 mrcnn_jaccard_coef 0.8568256859842879
yolo_dice_c 0.850828729281768,  unet_dice_c 0.9345191313340228 mrcnn_dice_c 0.9228929699236594
dataset/mrcnn_format/test/804892-5_17.jpg
Yolo Logs
(416, 416, 3)
Found 2 boxes for img
background 0.02 (97, 152) (232, 298)
background 0.98 (95, 93) (223, 236)
Time spent: 0.025sec


yoloIOU 0.4719922065270336,  unet_iou 0.9288757784333006 maskrcnn_iou 0.9107416444419225
yolo_jaccard_coef 0.35332151038202914,  unet_jaccard_coef 0.8845660538691749 mrcnn_jaccard_coef 0.7325839995191441
yolo_dice_c 0.5221545769745284,  unet_dice_c 0.9387477313974592 mrcnn_dice_c 0.8456548135299219
dataset/mrcnn_format/test/791552-5_20.jpg
Yolo Logs
(416, 416, 3)
Found 4 boxes for img
background 0.03 (113, 134) (204, 227)
background 0.03 (76, 2) (242, 178)
background 0.03 (102, 153) (268, 307)
background 0.88 (94

yoloIOU 0.21368171824792828,  unet_iou 0.8896723351306511 maskrcnn_iou 0.6754457117316957
yolo_jaccard_coef 0.1957298765667445,  unet_jaccard_coef 0.9021803619104123 mrcnn_jaccard_coef 0.9049934640522875
yolo_dice_c 0.32738142686329214,  unet_dice_c 0.9485749931771219 mrcnn_dice_c 0.9501276315428321
dataset/mrcnn_format/test/802346-5_23.jpg
Yolo Logs
(416, 416, 3)
Found 2 boxes for img
background 0.01 (67, 0) (252, 177)
background 0.06 (82, 82) (229, 237)
Time spent: 0.025sec


dataset/mrcnn_format/test/785256-5_18.jpg
Yolo Logs
(416, 416, 3)
Found 3 boxes for img
background 0.02 (85, 158) (243, 305)
background 0.04 (136, 113) (273, 259)
background 0.94 (88, 92) (225, 240)
Time spent: 0.027sec


yoloIOU 0.4590524950813808,  unet_iou 0.8534995977473854 maskrcnn_iou 0.9588826114249843
yolo_jaccard_coef 0.3436988543371522,  unet_jaccard_coef 0.8699815219217202 mrcnn_jaccard_coef 0.894610261470153
yolo_dice_c 0.5115712545676004,  unet_dice_c 0.9304707150556953 mrcnn_dice_c 0.94437392181753

yoloIOU 0.18877650374001992,  unet_iou 0.9538393645189762 maskrcnn_iou 0.38494368701837584
yolo_jaccard_coef 0.1614526361846423,  unet_jaccard_coef 0.8786164688427299 mrcnn_jaccard_coef 0.7125147213785409
yolo_dice_c 0.27801845922019214,  unet_dice_c 0.9353867416950491 mrcnn_dice_c 0.8321268278557985
dataset/mrcnn_format/test/811641-5_12.jpg
Yolo Logs
(416, 416, 3)
Found 3 boxes for img
background 0.01 (92, 116) (222, 244)
background 0.04 (151, 6) (307, 171)
background 0.80 (56, 62) (217, 215)
Time spent: 0.027sec


yoloIOU 0.5350869039873595,  unet_iou 0.879170171012055 maskrcnn_iou 0.9276108358535486
yolo_jaccard_coef 0.49798663468128856,  unet_jaccard_coef 0.8948423406059671 mrcnn_jaccard_coef 0.8310209496434692
yolo_dice_c 0.6648746032199948,  unet_dice_c 0.9445032142566523 mrcnn_dice_c 0.9077132075471698
dataset/mrcnn_format/test/799309-5_5.jpg
Yolo Logs
(416, 416, 3)
Found 3 boxes for img
background 0.01 (59, 35) (219, 189)
background 0.03 (91, 147) (238, 302)
background 0.95 (82

yoloIOU 0.4230161935886608,  unet_iou 0.880792585490572 maskrcnn_iou 0.24293858512568023
yolo_jaccard_coef 0.31613193994535366,  unet_jaccard_coef 0.7463913105616693 mrcnn_jaccard_coef 0.8332708177044261
yolo_dice_c 0.48039551408269837,  unet_dice_c 0.8547812921968984 mrcnn_dice_c 0.9090537084398977
dataset/mrcnn_format/test/813983-5_23.jpg
Yolo Logs
(416, 416, 3)
Found 3 boxes for img
background 0.07 (85, 112) (193, 204)
background 0.53 (81, 111) (237, 307)
background 0.94 (69, 79) (212, 236)
Time spent: 0.027sec


dataset/mrcnn_format/test/788123-5_13.jpg
Yolo Logs
(416, 416, 3)
Found 2 boxes for img
background 0.03 (77, 23) (227, 164)
background 0.99 (90, 89) (223, 228)
Time spent: 0.025sec


yoloIOU 0.35294820479120614,  unet_iou 0.7934865900383141 maskrcnn_iou 0.7870207709000724
yolo_jaccard_coef 0.3907254952405454,  unet_jaccard_coef 0.7886594470975987 mrcnn_jaccard_coef 0.6759768849752339
yolo_dice_c 0.5619016787679786,  unet_dice_c 0.881844163658243 mrcnn_dice_c 0.8066661193662

In [ ]:
print(f"Unet : Average Jacard Coef for Testing Imges : {sum(UNET_jacard_coef)/len(UNET_jacard_coef)}")
print(f"Unet : Average IOU for Testing Imges : {sum(UNET_IOU)/len(UNET_IOU)}")
print(f"Unet : Average Dice-Coef for Testing Imges : {sum(UNET_dice_coef)/len(UNET_dice_coef)}")

print("\n\n")

print(f"YOLO : Average Jacard Coef for Testing Imges : {sum(YOLO_jacard_coef)/len(YOLO_jacard_coef)}")
print(f"YOLO : Average IOU for Testing Imges : {sum(YOLO_IOU)/len(YOLO_IOU)}")
print(f"YOLO : Average Dice-Coef for Testing Imges : {sum(YOLO_dice_coef)/len(YOLO_dice_coef)}")

print("\n\n")

print(f"MRCNN : Average Jacard Coef for Testing Imges : {sum(MRCNN_jacard_coef)/len(MRCNN_jacard_coef)}")
print(f"MRCNN : Average IOU for Testing Imges : {sum(MRCNN_IOU)/len(MRCNN_IOU)}")
print(f"MRCNN : Average Dice-Coef for Testing Imges : {sum(MRCNN_dice_coef)/len(MRCNN_dice_coef)}")


In [ ]:
import plotly.graph_objects as go


# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=images_processed, y=UNET_jacard_coef,
                    mode='lines+markers',
                    name='Unet'))
fig.add_trace(go.Scatter(x=images_processed, y=YOLO_jacard_coef,
                    mode='lines+markers',
                    name='Yolo'))
fig.add_trace(go.Scatter(x=images_processed, y=MRCNN_jacard_coef,
                    mode='lines+markers', name='MRCNN'))


fig.update_layout(title='Jacard coefficient',
                   xaxis_title='Jaccard Coef',
                   yaxis_title='Number Of Images')

fig.show()

In [ ]:
import plotly.graph_objects as go


# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=images_processed, y=UNET_IOU,
                    mode='lines+markers',
                    name='Unet'))
fig.add_trace(go.Scatter(x=images_processed, y=YOLO_IOU,
                    mode='lines+markers',
                    name='Yolo'))
fig.add_trace(go.Scatter(x=images_processed, y=MRCNN_IOU,
                    mode='lines+markers', name='MRCNN'))


fig.update_layout(title='Intersection over Union of Masks',
                   xaxis_title='IOU',
                   yaxis_title='Number Of Images')

fig.show()

In [ ]:
import plotly.graph_objects as go


# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=images_processed, y=UNET_dice_coef,
                    mode='lines+markers',
                    name='Unet'))
fig.add_trace(go.Scatter(x=images_processed, y=YOLO_dice_coef,
                    mode='lines+markers',
                    name='Yolo'))
fig.add_trace(go.Scatter(x=images_processed, y=MRCNN_dice_coef,
                    mode='lines+markers', name='MRCNN'))


fig.update_layout(title='Intersection over Union of Masks',
                   xaxis_title='IOU',
                   yaxis_title='Number Of Images')

fig.show()

In [ ]:
import plotly.graph_objects as go
models=['Unet', 'Yolo', 'MRCNN']

fig = go.Figure(data=[
    go.Bar(name='Jacard', x=models, y=[sum(UNET_jacard_coef)/len(UNET_jacard_coef), sum(YOLO_jacard_coef)/len(YOLO_jacard_coef), sum(MRCNN_jacard_coef)/len(MRCNN_jacard_coef)]),
    go.Bar(name='Dice', x=models, y=[sum(UNET_dice_coef)/len(UNET_dice_coef), sum(YOLO_dice_coef)/len(YOLO_dice_coef), sum(MRCNN_dice_coef)/len(MRCNN_dice_coef)]),
    go.Bar(name='IOU', x=models, y=[sum(UNET_IOU)/len(UNET_IOU), sum(YOLO_IOU)/len(YOLO_IOU) , sum(MRCNN_dice_coef)/len(MRCNN_dice_coef)])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()